# OLD

In [ ]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import re

In [ ]:
df = pd.read_csv('full-lenta-lemmas.csv', sep=';', index_col=0)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.shape

In [ ]:
df.head(1)

In [ ]:
df['lemmas'] = df['lemmas'].apply(lambda x: x[1:-1].split(','))

In [ ]:
def clear_list_from_shit(lst):
    
    for i in range(len(lst)):
        lst[i] = re.sub(r"'",'', lst[i])
        lst[i] = lst[i].replace(' ', '')

In [ ]:
for i in tqdm(range(len(df['lemmas']))):
    
    clear_list_from_shit(df['lemmas'][i])

In [ ]:
def day_lemmas_sum(x, y):
    
    return x[x['date'] == y]['lemmas'].sum()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

dates = df['date'].unique()
df_days = pd.DataFrame(columns = {'date', 'day_lemmas'})

for i in tqdm(range(len(dates))):
    
    text = day_lemmas_sum(df, dates[i])
    new_row = {'date':dates[i], 'day_lemmas':text}
    df_days= df_days.append(new_row, ignore_index = True)

In [ ]:
dates[10]

In [ ]:
len(df_days['day_lemmas'][10])

In [ ]:
len(df[df['date'] == dates[10]]['lemmas'].sum())

In [ ]:
df_days['date'] = pd.to_datetime(df_days['date'])

In [ ]:
import datetime

x = pd.to_datetime(df_days['date'].max()) - datetime.timedelta(days=365)
x

In [ ]:
df_last_365_days = df_days[df_days['date'] > x]

In [ ]:
# df_last_365_days.to_csv('df_last_365_days.csv', header=True, sep=';')

# Загрузка и обработка датасета

In [1]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import re
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('df_last_365_days.csv', sep=';', index_col=0)
df.reset_index(drop=True, inplace=True)

# Делаем листы листами

df['day_lemmas'] = df['day_lemmas'].apply(lambda x: x[1:-1].split(','))

def clear_list_from_shit(lst):
    
    for i in range(len(lst)):
        lst[i] = re.sub(r"'",'', lst[i])
        lst[i] = lst[i].replace(' ', '')
        
for i in tqdm(range(len(df['day_lemmas']))):
    clear_list_from_shit(df['day_lemmas'][i])

    
# Столбец с текстом STR
    
df['day_texts'] = ""

for i in range(len(df)):
    df['day_texts'][i] = " ".join(df['day_lemmas'][i])
    
    
# Соединяем по 10 токенов в предложения

def stroka_to_predl_list(strka):
    
    hmch = round(len(strka) / 10)
    output = []
    x = 0
    y = 10
    
    for i in range(hmch):
        
        ten_words = strka[x:y]
        cnct = " ".join(ten_words)
        output.append([cnct])
        x += 10
        y += 10
    
    return output

df['day_predls'] = ""

for i in tqdm(range(len(df))):
    
    predls = stroka_to_predl_list(df['day_lemmas'][i])
    df['day_predls'][i] = predls

  0%|          | 0/362 [00:00<?, ?it/s]

  0%|          | 0/362 [00:00<?, ?it/s]

In [2]:
df

,date,day_lemmas,day_texts,day_predls
0,2021-05-13,"[бедный, народ, предложить, доплачивать, жизнь...",бедный народ предложить доплачивать жизнь лес ...,[[бедный народ предложить доплачивать жизнь ле...
1,2021-05-14,"[медведев, назвать, охота, ведьма, арест, медв...",медведев назвать охота ведьма арест медведчук ...,[[медведев назвать охота ведьма арест медведчу...
2,2021-05-15,"[spacex, запустить, starship, орбита, вокруг, ...",spacex запустить starship орбита вокруг земля ...,[[spacex запустить starship орбита вокруг земл...
3,2021-05-16,"[хамас, снова, выпустить, ракета, сторона, изр...",хамас снова выпустить ракета сторона израиль п...,[[хамас снова выпустить ракета сторона израиль...
4,2021-05-17,"[спч, призвать, посмертно, наградить, спасать,...",спч призвать посмертно наградить спасать ребён...,[[спч призвать посмертно наградить спасать реб...
...,...,...,...,...
357,2022-05-08,"[калининград, назвать, подрывать, сила, нато, ...",калининград назвать подрывать сила нато авиано...,[[калининград назвать подрывать сила нато авиа...
358,2022-05-09,"[матвиенко, призвать, пресекать, распространен...",матвиенко призвать пресекать распространение н...,[[матвиенко призвать пресекать распространение...
359,2022-05-10,"[российский, дипломат, оборвать, эфир, sky, ne...",российский дипломат оборвать эфир sky news из-...,[[российский дипломат оборвать эфир sky news и...
360,2022-05-11,"[турин, стартовать, евровидение-, международны...",турин стартовать евровидение- международный пе...,[[турин стартовать евровидение- международный ...


# Алгоритм

In [3]:
import pandas as pd
import re 
from tqdm.notebook import tqdm
import numpy as np 
from datetime import datetime
import json
from IPython.core.display import display, HTML
import pickle



import pymorphy2
morph = pymorphy2.MorphAnalyzer()


# Сам алгоритм 

from collections import Counter, defaultdict
import re

def merge_vocab(pair: tuple, v_in: dict) -> dict:
    """Step 3. Merge all occurrences of the most frequent pair"""
    
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    xx = '_'.join(pair)
    
    for word in v_in:
        # replace most frequent pair in all vocabulary
        w_out = p.sub(xx, word)
        v_out[w_out] = v_in[word]

    return v_out

def get_stats(vocab: dict) -> dict:
    """Step 2. Get counts of pairs of consecutive symbols"""

    pairs = defaultdict(int)

    for word, frequency in vocab.items():

        symbols = word.split(' ')

        symbols = [i for i in symbols if not '</w>' in i ]

        #symbols = [i for i in symbols if i not in stopwords ]

        # Counting up occurrences of pairs
        for i in range(len(symbols) - 1):

            k = (symbols[i], symbols[i + 1])

            # Пропускаем шумные токены в обработке 
            if 'mentula' in ' '.join(k):
                continue
            
            pairs[k] += frequency

    return pairs

def build_vocab(corpus: str) -> dict:
    """Step 1. Build vocab from text corpus"""

    # Separate each char in word by space and add mark end of token
    tokens = [" ".join(word.split(' ')) + " </w>" for word in corpus]
    
    # Count frequency of tokens in corpus
    vocab = Counter(tokens)  

    return vocab

In [4]:
# Что подаю на вход

df['day_predls'][0][:5]

[['бедный народ предложить доплачивать жизнь лес действовать мировой система возмещение'],
 ['нанести вред экология неэффективный наносить вред окружающий среда вывод прислать'],
 ['эксперт благотворительный организация сеть третий мир изменение сложиться ситуация частность'],
 ['предлагать начать платить народ бедный страна жизнь лес охрана флора'],
 ['фауна это сообщать би-би-си настоящий момент ведущий экономика мир ежегодно']]

In [22]:
len(df['day_predls'])

362

In [24]:
df['date'][361]

'2022-05-12'

In [25]:
new_day_data_chunk = df['day_predls'][361]

In [26]:
new_new_day_data_chunk = []

for chunk in tqdm(new_day_data_chunk):

    text_for_bpe = [i for i in chunk]

    text_for_bpe = [i for i in text_for_bpe if len(i.split(' ')) > 2]

    # Свернем с сохранением порядка 
    text_for_bpe = list(dict.fromkeys(text_for_bpe).keys())

    text = text_for_bpe
    
    vocab = build_vocab(text)

    num_merges = 200  # Hyperparameter

    out = []
    
    for i in range(num_merges):

            pairs = get_stats(vocab)  # Step 2

            if not pairs:
                break

            best = max(pairs, key=pairs.get)

            #print(best, pairs[best])

            out += [(best, pairs[best])]

            vocab = merge_vocab(best, vocab)
            
    out = [(' '.join(i[0]).replace('_', ' '), i[1]) for i in out]
    new_new_day_data_chunk.append([out , chunk]) 

  0%|          | 0/372 [00:00<?, ?it/s]

In [27]:
len(new_new_day_data_chunk)

372

In [28]:
days = pd.DataFrame()

In [29]:
days['word'] = ""
days['freq'] = ""

In [30]:
for i in range(len(new_new_day_data_chunk)):
    
    new_row = {'word':new_new_day_data_chunk[i][0][0][0], 'freq':new_new_day_data_chunk[i][0][0][1]}
    days= days.append(new_row, ignore_index = True)


In [31]:
days.sort_values(by='freq', ascending=False).iloc[:10]

,word,freq
303,президент бразилия,2
273,турция усомниться,2
42,кубок италия,2
222,долговой бремя,2
245,фамилия хансный,1
253,вне украина,1
252,россия wp,1
251,разрезать свадебный,1
250,жених напугать,1
249,гостья свадьба,1


In [ ]:
import re
from collections import Counter, defaultdict


def build_vocab(corpus: str) -> dict:
    """Step 1. Build vocab from text corpus"""

    # Separate each char in word by space and add mark end of token
    tokens = [" ".join(word) + " </w>" for word in corpus.split()]
    
    # Count frequency of tokens in corpus
    vocab = Counter(tokens)  

    return vocab


def get_stats(vocab: dict) -> dict:
    """Step 2. Get counts of pairs of consecutive symbols"""

    pairs = defaultdict(int)
    for word, frequency in vocab.items():
        symbols = word.split()

        # Counting up occurrences of pairs
        for i in range(len(symbols) - 1):
            pairs[symbols[i], symbols[i + 1]] += frequency

    return pairs


def merge_vocab(pair: tuple, v_in: dict) -> dict:
    """Step 3. Merge all occurrences of the most frequent pair"""
    
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        # replace most frequent pair in all vocabulary
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]

    return v_out


In [ ]:
vocab = build_vocab(df['day_texts'][0])  # Step 1

num_merges = 500  # Hyperparameter

In [ ]:
for i in range(num_merges):

    pairs = get_stats(vocab)  # Step 2

    if not pairs:
        break

    # step 3
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)

In [ ]:
best

In [ ]:
sorted(vocab.items(), key=lambda x: x[1], reverse=True)